# Emojify！

欢迎来到第2周的第二次作业。您将使用单词向量表示来构建Emojifier。

您是否曾经想过让短信更具表现力？您的emojifier应用程序将帮助您做到这一点。因此，与其写“恭喜升级！让我们喝咖啡和聊天。爱你！” emojifier可以自动将其转换为“恭喜促销！👍让我们喝咖啡聊天。☕️爱你！❤️”

您将实现一个模型，该模型输入一个句子（例如“让我们今晚去看棒球比赛！”），并找到最适合与该句子搭配使用的表情符号（⚾️）。在许多表情符号界面中，您需要记住❤️是“心”符号而不是“爱”符号。但是使用单词向量，您会看到，即使您的训练集仅将几个单词与特定表情符号明确关联，您的算法也能够将测试集中的单词归纳并关联到同一表情符号，即使这些单词没有甚至不会出现在训练集中。这样，即使使用很小的训练集，您也可以建立从句子到表情符号的准确分类器映射。

在本练习中，您将从使用单词嵌入的基线模型（Emojifier-V1）开始，然后构建一个进一步包含LSTM的更复杂的模型（Emojifier-V2）。

让我们开始吧！运行以下单元格以加载要使用的软件包。

In [ ]:
import numpy as np
from emo_utils import *
import emoji
import matplotlib.pyplot as plt

%matplotlib inline

## 1 - 基线型号：Emojifier-V1

### 1.1 - 数据集EMOJISET

让我们从构建一个简单的基线分类器开始。

您有一个很小的数据集（X，Y），其中：
- X包含127个句子（字符串）
- Y包含一个介于0到4之间的整数标签，对应于每个句子的表情符号

<img src="images/data_set.png" style="width:700px;height:300px;">
<caption><center> **图1 **：EMOJISET-具有5个类的分类问题。 这里给出一些句子的例子。 </center></caption>

让我们使用下面的代码加载数据集。 我们将数据集分为训练（127个示例）和测试（56个示例）之间。

In [ ]:
X_train, Y_train = read_csv('data/train_emoji.csv')
X_test, Y_test = read_csv('data/tesss.csv')

In [ ]:
maxLen = len(max(X_train, key=len).split())

运行以下单元格以打印来自X_train的句子和来自Y_train的相应标签。 更改`index`以查看不同的示例。 由于iPython笔记本使用的字体，心脏表情符号的颜色可能是黑色，而不是红色。

In [ ]:
index = 1
print(X_train[index], label_to_emoji(Y_train[index]))

### 1.2-Emojifier-V1概述

在这一部分中，您将实现一个称为“ Emojifier-v1”的基线模型。

<center>
<img src="images/image_1.png" style="width:900px;height:300px;">
<caption><center> **Figure 2**: Baseline model (Emojifier-V1).</center></caption>
</center>

模型的输入是与句子相对应的字符串（例如，“我爱你”。在代码中，输出将是形状为（1,5）的概率矢量，然后将其传递到argmax层中以提取 最有可能的表情符号输出的索引。

为了使我们的标签成为适合训练softmax分类器的格式，让我们将$Y$从当前形状的当前形状 $(m, 1)$转换为“单热表示” $(m, 5)$，其中每个 row是一个热门向量，提供了一个示例的标签，您可以使用下一个代码截取器来实现。 在这里，`Y_oh`在变量名 `Y_oh_train`和“ `Y_oh_test`中代表“ Y-one-hot”：

In [ ]:
Y_oh_train = convert_to_one_hot(Y_train, C = 5)
Y_oh_test = convert_to_one_hot(Y_test, C = 5)

让我们看看 `convert_to_one_hot()`做了什么。 随时更改`index`以输出不同的值。

In [ ]:
index = 50
print(Y_train[index], "is converted into one hot", Y_oh_train[index])

现在，所有数据都已准备好送入Emojify-V1模型。 让我们实现模型！

### 1.3 - 实施Emojifier-V1

如图（2）所示，第一步是将输入句子转换为单词向量表示形式，然后将其平均在一起。 与之前的练习类似，我们将使用预训练的50维GloVe嵌入。 运行以下单元格以加载`word_to_vec_map`，其中包含所有矢量表示。

In [ ]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('data/glove.6B.50d.txt')

您已加载：
- `word_to_index`：从单词到词汇表中的索引的字典映射（400,001个单词，有效索引范围是0到400,000）
- `index_to_word`：字典从索引到词汇表中对应词的映射
- `word_to_vec_map`：将单词映射到其GloVe矢量表示的字典。

运行以下单元格以检查其是否有效。

In [ ]:
word = "cucumber"
index = 289846
print("the index of", word, "in the vocabulary is", word_to_index[word])
print("the", str(index) + "th word in the vocabulary is", index_to_word[index])

**练习**：实现`sentence_to_avg（）`。 您将需要执行两个步骤：
1. 将每个句子转换为小写，然后将句子拆分为单词列表。`X.lower()`和`X.split()`可能有用。
2. 对于句子中的每个单词，请访问其GloVe表示。 然后，将所有这些值取平均值。

In [ ]:
# GRADED FUNCTION: sentence_to_avg

def sentence_to_avg(sentence, word_to_vec_map):
    """
    Converts a sentence (string) into a list of words (strings). Extracts the GloVe representation of each word
    and averages its value into a single vector encoding the meaning of the sentence.
    
    Arguments:
    sentence -- string, one training example from X
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    
    Returns:
    avg -- average vector encoding information about the sentence, numpy-array of shape (50,)
    """
    
    ### START CODE HERE ###
    # Step 1: Split sentence into list of lower case words (≈ 1 line)
    words = None

    # Initialize the average word vector, should have the same shape as your word vectors.
    avg = None
    
    # Step 2: average the word vectors. You can loop over the words in the list "words".
    for w in None:
        avg += None
    avg = None
    
    ### END CODE HERE ###
    
    return avg

In [ ]:
avg = sentence_to_avg("Morrocan couscous is my favorite dish", word_to_vec_map)
print("avg = ", avg)

**预期输出t**:

<table>
    <tr>
        <td>
            **avg= **
        </td>
        <td>
           [-0.008005    0.56370833 -0.50427333  0.258865    0.55131103  0.03104983
 -0.21013718  0.16893933 -0.09590267  0.141784   -0.15708967  0.18525867
  0.6495785   0.38371117  0.21102167  0.11301667  0.02613967  0.26037767
  0.05820667 -0.01578167 -0.12078833 -0.02471267  0.4128455   0.5152061
  0.38756167 -0.898661   -0.535145    0.33501167  0.68806933 -0.2156265
  1.797155    0.10476933 -0.36775333  0.750785    0.10282583  0.348925
 -0.27262833  0.66768    -0.10706167 -0.283635    0.59580117  0.28747333
 -0.3366635   0.23393817  0.34349183  0.178405    0.1166155  -0.076433
  0.1445417   0.09808667]
        </td>
    </tr>
</table>

#### 模型

现在，您已经完成了完成`model（）`函数的所有工作。 使用`sentence_to_avg（）`之后，您需要使平均值通过正向传播，计算成本，然后向后传播以更新softmax的参数。

**练习**：实现图（2）中描述的`model（）`函数。 假设$Yoh$（"Y one hot"）是输出标签的一热编码，则在前向传递中需要实现的公式和计算交叉熵代价的公式为：
$$ z^{(i)} = W \times avg^{(i)} + b$$
$$ a^{(i)} = softmax(z^{(i)})$$
$$ \mathcal{L}^{(i)} = - \sum_{k = 0}^{n_y - 1} Yoh^{(i)}_k * log(a^{(i)}_k)$$



有可能提出一个更有效的矢量化实现。 但是，由于我们一直在使用for循环将句子一次转换为avg^{(i)}表示形式，所以这次我们不打扰。

我们为您提供了一个函数`softmax()`。

In [ ]:
# GRADED FUNCTION: model

def model(X, Y, word_to_vec_map, learning_rate = 0.01, num_iterations = 400):
    """
    Model to train word vector representations in numpy.
    
    Arguments:
    X -- input data, numpy array of sentences as strings, of shape (m, 1)
    Y -- labels, numpy array of integers between 0 and 7, numpy-array of shape (m, 1)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    learning_rate -- learning_rate for the stochastic gradient descent algorithm
    num_iterations -- number of iterations
    
    Returns:
    pred -- vector of predictions, numpy-array of shape (m, 1)
    W -- weight matrix of the softmax layer, of shape (n_y, n_h)
    b -- bias of the softmax layer, of shape (n_y,)
    """
    
    np.random.seed(1)

    # Define number of training examples
    m = Y.shape[0]                          # number of training examples
    n_y = 5                                 # number of classes  
    n_h = 50                                # dimensions of the GloVe vectors 
    
    # Initialize parameters using Xavier initialization
    W = np.random.randn(n_y, n_h) / np.sqrt(n_h)
    b = np.zeros((n_y,))
    
    # Convert Y to Y_onehot with n_y classes
    Y_oh = convert_to_one_hot(Y, C = n_y) 
    
    # Optimization loop
    for t in range(num_iterations):                       # Loop over the number of iterations
        for i in range(m):                                # Loop over the training examples
            
            ### START CODE HERE ### (≈ 4 lines of code)
            # Average the word vectors of the words from the j'th training example
            avg = None

            # Forward propagate the avg through the softmax layer
            z = None
            a = None

            # Compute cost using the j'th training label's one hot representation and "A" (the output of the softmax)
            cost = None
            ### END CODE HERE ###
            
            # Compute gradients 
            dz = a - Y_oh[i]
            dW = np.dot(dz.reshape(n_y,1), avg.reshape(1, n_h))
            db = dz

            # Update parameters with Stochastic Gradient Descent
            W = W - learning_rate * dW
            b = b - learning_rate * db
        
        if t % 100 == 0:
            print("Epoch: " + str(t) + " --- cost = " + str(cost))
            pred = predict(X, Y, W, b, word_to_vec_map)

    return pred, W, b

In [ ]:
print(X_train.shape)
print(Y_train.shape)
print(np.eye(5)[Y_train.reshape(-1)].shape)
print(X_train[0])
print(type(X_train))
Y = np.asarray([5,0,0,5, 4, 4, 4, 6, 6, 4, 1, 1, 5, 6, 6, 3, 6, 3, 4, 4])
print(Y.shape)

X = np.asarray(['I am going to the bar tonight', 'I love you', 'miss you my dear',
 'Lets go party and drinks','Congrats on the new job','Congratulations',
 'I am so happy for you', 'Why are you feeling bad', 'What is wrong with you',
 'You totally deserve this prize', 'Let us go play football',
 'Are you down for football this afternoon', 'Work hard play harder',
 'It is suprising how people can be dumb sometimes',
 'I am very disappointed','It is the best day in my life',
 'I think I will end up alone','My life is so boring','Good job',
 'Great so awesome'])

print(X.shape)
print(np.eye(5)[Y_train.reshape(-1)].shape)
print(type(X_train))


运行下一个单元格以训练您的模型并学习softmax参数（W，b）。

In [ ]:
pred, W, b = model(X_train, Y_train, word_to_vec_map)
print(pred)

**预期输出** (on a subset of iterations):

<table>
    <tr>
        <td>
            **Epoch: 0**
        </td>
        <td>
           cost = 1.95204988128
        </td>
        <td>
           Accuracy: 0.348484848485
        </td>
    </tr>


<tr>
        <td>
            **Epoch: 100**
        </td>
        <td>
           cost = 0.0797181872601
        </td>
        <td>
           Accuracy: 0.931818181818
        </td>
    </tr>
    
<tr>
        <td>
            **Epoch: 200**
        </td>
        <td>
           cost = 0.0445636924368
        </td>
        <td>
           Accuracy: 0.954545454545
        </td>
    </tr> 
    <tr>
        <td>
            **Epoch: 300**
        </td>
        <td>
           cost = 0.0343226737879
        </td>
        <td>
           Accuracy: 0.969696969697
        </td>
    </tr>
</table>

大！ 您的模型在训练集上具有很高的准确性。 现在让我们看看它如何在测试集上运行。

### 1.4 - 检查测试仪性能


In [ ]:
print("Training set:")
pred_train = predict(X_train, Y_train, W, b, word_to_vec_map)
print('Test set:')
pred_test = predict(X_test, Y_test, W, b, word_to_vec_map)

**预期输出**:

<table>
    <tr>
        <td>
            **Train set accuracy**
        </td>
        <td>
           97.7
        </td>
    </tr>
    <tr>
        <td>
            **Test set accuracy**
        </td>
        <td>
           85.7
        </td>
    </tr>
</table>

假设有5个类别，那么随机猜测的准确率将达到20％。 在仅训练了127个示例之后，这是相当不错的表现。

在训练集中，算法看到带有标签❤️的句子“ * I love you *”。 但是，您可以检查单词“ adore”是否没有出现在训练集中。 不过，让我们看看如果您写“ *我崇拜您*”会发生什么。


In [ ]:
X_my_sentences = np.array(["i adore you", "i love you", "funny lol", "lets play with a ball", "food is ready", "not feeling happy"])
Y_my_labels = np.array([[0], [0], [2], [1], [4],[3]])

pred = predict(X_my_sentences, Y_my_labels , W, b, word_to_vec_map)
print_predictions(X_my_sentences, pred)

惊人！ 由于* adore *具有与* love *类似的嵌入方式，因此该算法可以正确地推广到甚至从未见过的单词。 * heart *，* dear *，* beloved *或* adore *之类的词具有类似于* love *的嵌入向量，因此也可能起作用---随意修改上面的输入并尝试各种输入句子 。 效果如何？

请注意，尽管这样并不能使“感到不开心”正确。 该算法忽略单词顺序，因此不善于理解“不高兴”之类的短语。

打印混淆矩阵还可以帮助您了解哪些类别对您的模型而言更加困难。 混淆矩阵显示算法将标签为一个类别（“实际”类别）的示例错误地标记为具有不同类别（“预测”类别）的示例。


In [ ]:
print(Y_test.shape)
print('           '+ label_to_emoji(0)+ '    ' + label_to_emoji(1) + '    ' +  label_to_emoji(2)+ '    ' + label_to_emoji(3)+'   ' + label_to_emoji(4))
print(pd.crosstab(Y_test, pred_test.reshape(56,), rownames=['Actual'], colnames=['Predicted'], margins=True))
plot_confusion_matrix(Y_test, pred_test)

<font color='blue'>
**这部分您应该记住**：
    
- 即使有127个训练示例，您也可以获得合理的Emojifying模型。 这是由于单词向量为您提供的泛化能力。
- Emojify-V1在*“这部电影不好，也不愉快” *等句子上表现不佳，因为它不理解单词的组合-只是将所有单词的嵌入向量平均在一起，而没有注意 单词的顺序。 在下一部分中，您将构建一个更好的算法。

## 2 - Emojifier-V2：在Keras中使用LSTM：

让我们建立一个LSTM模型作为输入单词序列。 此模型将能够考虑单词顺序。 Emojifier-V2将继续使用预训练的单词嵌入来表示单词，但会将其输入到LSTM中，LSTM的工作是预测最合适的表情符号。

运行以下单元格以加载Keras软件包。

In [ ]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)

### 2.1-模型概述

这是您将实现的Emojifier-v2：

<img src="images/emojifier-v2.png" style="width:700px;height:400px;"> <br>
<caption><center> **Figure 3**: Emojifier-V2. A 2-layer LSTM sequence classifier. </center></caption>



### 2.2 Keras和迷你批处理

在本练习中，我们想使用迷你批次训练Keras。但是，大多数深度学习框架要求同一迷你批中的所有序列都具有相同的长度。这就是向量化工作的原理：如果您有3个单词的句子和4个单词的句子，那么它们所需的计算是不同的（一个LSTM需要3个步骤，一个LSTM需要4个步骤），所以这是不可能的同时做他们两个。

常见的解决方案是使用填充。具体来说，设置最大序列长度，并将所有序列填充为相同的长度。例如，最大序列长度为20，我们可以用“ 0”填充每个句子，以便每个输入句子的长度为20。因此，句子“我爱你”将表示为$(e_{i}, e_{love}, e_{you}, \vec{0}, \vec{0}, \ldots, \vec{0})$。在此示例中，任何长度超过20个单词的句子都必须被截断。选择最大序列长度的一种简单方法是仅选择训练集中最长句子的长度。

### 2.3-嵌入层

在Keras中，嵌入矩阵表示为“层”，并将正整数（对应于单词的索引）映射为固定大小的密集向量（嵌入向量）。可以使用预训练的嵌入对其进行训练或初始化。在这一部分中，您将学习如何在Keras中创建[Embedding()](https://keras.io/layers/embeddings/)层，并使用之前在笔记本中加载的GloVe 50维向量对其进行初始化。因为我们的训练集很小，所以我们不会更新单词嵌入，而是将其值保持不变。但是在下面的代码中，我们将向您展示Keras如何允许您训练或固定该层。

 `Embedding()` 层采用大小（批量大小，最大输入长度）的整数矩阵作为输入。如下图所示，这对应于转换为索引列表（句子）的句子。

<img src="images/embedding1.png" style="width:700px;height:250px;">
<caption><center> **图4 **：嵌入层。此示例显示了两个示例通过嵌入层的传播。两者都被零填充到 `max_len=5`的长度。表示的最终尺寸为`(2,max_len,50)`，因为我们使用的词嵌入为50维。</center></caption>

输入中的最大整数（即单词索引）应不大于词汇量。图层输出一个形状数组（批大小，最大输入长度，单词向量的维）。

第一步是将所有训练语句转换为索引列表，然后对所有这些列表进行零填充，以使它们的长度为最长句子的长度。

**练习**：实现以下功能，将X（字符串形式的句子数组）转换为与句子中单词相对应的索引数组。输出形状应使其可以赋予`Embedding（）`（如图4所示）。

In [ ]:
# GRADED FUNCTION: sentences_to_indices

def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    m = X.shape[0]                                   # number of training examples
    
    ### START CODE HERE ###
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = None
    
    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words =None
        
        # Initialize j to 0
        j = None
        
        # Loop over the words of sentence_words
        for w in None:
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            X_indices[i, j] = None
            # Increment j to j + 1
            j = None
            
    ### END CODE HERE ###
    
    return X_indices

运行以下单元格以检查`sentences_to_indices（）`的作用，并检查结果。

In [ ]:
X1 = np.array(["funny lol", "lets play baseball", "food is ready for you"])
X1_indices = sentences_to_indices(X1,word_to_index, max_len = 5)
print("X1 =", X1)
print("X1_indices =", X1_indices)

**预期输出**:

<table>
    <tr>
        <td>
            **X1 =**
        </td>
        <td>
           ['funny lol' 'lets play football' 'food is ready for you']
        </td>
    </tr>
    <tr>
        <td>
            **X1_indices =**
        </td>
        <td>
           [[ 155345.  225122.       0.       0.       0.] <br>
            [ 220930.  286375.  151266.       0.       0.] <br>
            [ 151204.  192973.  302254.  151349.  394475.]]
        </td>
    </tr>
</table>

让我们使用预先训练的单词向量在Keras中构建`Embedding()`层。 建立此层之后，您将把`sentences_to_indices()`的输出作为输入传递给它，而`Embedding()`层将返回句子的单词嵌入。

**练习**：实现`pretrained_embedding_layer（）`。 您将需要执行以下步骤：
1. 用正确的形状将嵌入矩阵初始化为零的numpy数组。
2. 使用从`word_to_vec_map`提取的所有词嵌入来填充嵌入矩阵。
3. 定义Keras嵌入层。 使用 [Embedding()](https://keras.io/layers/embeddings/)。 确保在调用`Embedding()`时通过设置 `trainable = False`来使该层不可训练。 如果要设置`trainable = True`，那么它将允许优化算法修改单词嵌入的值。
4. 将嵌入权重设置为等于嵌入矩阵


In [ ]:
# GRADED FUNCTION: pretrained_embedding_layer

def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    ### START CODE HERE ###
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = None
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        emb_matrix[index, :] = None

    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = None
    ### END CODE HERE ###

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [ ]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])

**预期输出**:

<table>
    <tr>
        <td>
            **weights[0][1][3] =**
        </td>
        <td>
           -0.3403
        </td>
    </tr>
</table>

## 2.3 Building the Emojifier-V2

现在让我们构建Emojifier-V2模型。 您将使用已构建的嵌入层来执行此操作，并将其输出提供给LSTM网络。

<img src="images/emojifier-v2.png" style="width:700px;height:400px;"> <br>
<caption><center> **Figure 3**: Emojifier-v2. A 2-layer LSTM sequence classifier. </center></caption>


**练习：**实现`Emojify_V2（）`，它会构建如图3所示的架构的Keras图。该模型将由`input_shape`定义的形状为(`m`, `max_len`, )的句子数组作为输入。 它应该输出形状为softmax的概率矢量(`m`, `C = 5`)。 您可能需要`Input(shape = ..., dtype = '...')`, [LSTM()](https://keras.io/layers/recurrent/#lstm), [Dropout()](https://keras.io/layers/core/#dropout), [Dense()](https://keras.io/layers/core/#dense), 和 [Activation()](https://keras.io/activations/).

In [ ]:
# GRADED FUNCTION: Emojify_V2

def Emojify_V2(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    ### START CODE HERE ###
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = None
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = None
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = None   
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = None
    # Add dropout with a probability of 0.5
    X = None
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = None
    # Add dropout with a probability of 0.5
    X = None
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = None
    # Add a softmax activation
    X = None
    
    # Create Model instance which converts sentence_indices into X.
    model = None
    
    ### END CODE HERE ###
    
    return model

运行以下单元格以创建您的模型并检查其摘要。 由于数据集中的所有句子均少于10个单词，因此我们选择`max_len = 10`。 您应该看到您的体系结构，它使用“ 20,223,927”参数，其中20,000,050（词嵌入）是不可训练的，其余223,877是不可训练的。 因为我们的词汇量有400,001个单词（有效索引从0到400,000），所以有400,001 \ * 50 = 20,000,050个不可训练的参数。

In [ ]:
model = Emojify_V2((maxLen,), word_to_vec_map, word_to_index)
model.summary()

像往常一样，在Keras中创建模型后，您需要对其进行编译并定义要使用的损耗，优化器和指标。 使用 `categorical_crossentropy` 损失，`adam` 优化器和`['accuracy']` 度量来编译模型：

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

现在该训练您的模型了。 您的Emojifier-V2 `model` 使用输入形状数组(`m`, `max_len`)，并输出形状概率矢量(`m`, `number of classes`)。 因此，我们必须将X_train（作为字符串的句子数组）转换为X_train_indices（作为单词索引列表的句子数组），并将Y_train（作为索引的标签）转换为Y_train_oh（作为一元向量的标签）。

In [ ]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 5)

在`X_train_indices`和`Y_train_oh`上拟合Keras模型。 我们将使用`epochs = 50`和`batch_size = 32`。

In [ ]:
model.fit(X_train_indices, Y_train_oh, epochs = 50, batch_size = 32, shuffle=True)

您的模型在训练集上的表现应接近**100％准确度**。 您获得的确切准确性可能有所不同。 运行以下单元格以在测试集上评估模型。

In [ ]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
Y_test_oh = convert_to_one_hot(Y_test, C = 5)
loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print()
print("Test accuracy = ", acc)

您应该获得80％到95％的测试精度。 运行下面的单元格以查看标签错误的示例。

In [ ]:
# This code allows you to see the mislabelled examples
C = 5
y_test_oh = np.eye(C)[Y_test.reshape(-1)]
X_test_indices = sentences_to_indices(X_test, word_to_index, maxLen)
pred = model.predict(X_test_indices)
for i in range(len(X_test)):
    x = X_test_indices
    num = np.argmax(pred[i])
    if(num != Y_test[i]):
        print('Expected emoji:'+ label_to_emoji(Y_test[i]) + ' prediction: '+ X_test[i] + label_to_emoji(num).strip())

现在，您可以按照自己的示例进行尝试。 在下面写下你自己的句子。

In [ ]:
# Change the sentence below to see your prediction. Make sure all the words are in the Glove embeddings.  
x_test = np.array(['not feeling happy'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
print(x_test[0] +' '+  label_to_emoji(np.argmax(model.predict(X_test_indices))))

以前，Emojify-V1模型没有正确标记“不高兴”，但是我们对Emojiy-V2的实施正确了。 （Keras的输出每次都是稍微随机的，因此您可能无法获得相同的结果。）由于训练集很小，因此当前模型在理解否定（例如“不高兴”）方面仍然不是很健壮。 有很多否定的例子。 但是，如果训练集更大，则LSTM模型在理解此类复杂句子方面将比Emojify-V1模型好得多。

### 恭喜！

您已经完成了此笔记本！ ❤️❤️❤️

<font color='blue'>
**您应该记住的**：
    
- 如果您的NLP任务的训练集很小，则使用单词嵌入可以大大帮助您的算法。单词嵌入使您的模型可以处理测试集中甚至可能没有出现在训练集中的单词。
- Keras（和大多数其他深度学习框架）中的训练序列模型需要一些重要的细节：
    - 要使用迷你批处理，需要填充序列，以便迷你批处理中的所有示例都具有相同的长度。
    - 可以使用预训练的值来初始化`Embedding（）`层。这些值可以是固定的，也可以在数据集中进一步训练。但是，如果您标记的数据集很小，则通常不值得尝试训练大量预训练的嵌入。
    - `LSTM（）`具有一个名为`return_sequences`的标志，用于确定您是要返回每个隐藏状态还是仅返回最后一个状态。
    - 您可以在`LSTM（）`之后立即使用`Dropout（）`来规范您的网络。

祝贺您完成此任务并构建了Emojifier。 我们希望您对这款笔记本所取得的成就感到满意！

# 😀😀😀😀😀😀




## 致谢

感谢Alison Darcy和Woebot团队对创建此任务的建议。 Woebot是聊天机器人的朋友，随时准备与您24/7通话。 作为Woebot技术的一部分，它使用单词嵌入来理解您所说的内容。 您可以通过访问http://woebot.io 来玩

<img src="images/woebot.png" style="width:600px;height:300px;">


